[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/joshmaglione/CS102-Jupyter/main?labpath=.%2F03_ndarray.ipynb) 

<a href="https://colab.research.google.com/github/joshmaglione/CS102-Jupyter/blob/main/03_ndarray.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 

[View on GitHub](https://github.com/joshmaglione/CS102-Jupyter/blob/main/03_ndarray.ipynb)

# 03 : Indexing, Reshaping, and Computing with `ndarray`

## Learning outcomes

- Use slicing, boolean masks, and advanced indexing with `ndarray`.
- Reshape arrays and reason about `shape`/axes.
- Compute summaries efficiently using vectorized operations.

From last time:
- NumPy arrays have type `ndarray`
- `ndarray`s are *homogeneous multi-dimensional* collections of data. 
- Some attributes of an `ndarray`:
  - `dtype` : the data type of the entries,
  - `ndim` : the number of dimensions, 
  - `shape` : the size of each dimension, 
  - `size` : the total size of the array

Additional attributes include:
- `itemsize` : the size (in bytes) of each array element, 
- `nbytes` : the total bytes used by the array.

## Some advantages of the `ndarray`

In [1]:
import numpy as np

### Memory efficiency

Let's look at the size (in bytes) of an instance of `ndarray`.

In [ ]:
x3 = np.random.randint(10, size=(3, 2, 5)) 
print(x3)

In [ ]:
print(f"itemsize: {x3.itemsize} bytes")
print(f"nbytes: {x3.nbytes} bytes")

Significantly fewer bytes. 

In general, `nbytes` is equal to `itemsize` times `size`.

In [ ]:
x3.itemsize * x3.size == x3.nbytes

Let's compare this with a list in Python, and let's make them larger to more easily see the difference.

In [ ]:
from sys import getsizeof

# Size of our lists
N = 10000

# Create a list of N elements 
S = range(N)

# Get the size of every element and the container
S_size = sum(getsizeof(x) for x in S) + getsizeof(S)

# Create a Numpy array of N elements 
D = np.arange(N)

print(f"Size of the Python list + container:       {S_size} bytes")
print(f"Size of one element in the NumPy array:    {D.itemsize} bytes")
print(f"Size of the entire NumPy array:            {D.nbytes} bytes")

### Iterating through lists

Let's do a simple operation with `list` and `ndarray`

In [6]:
# Create lists of size N
N = 10000
Xpy = range(N)
Ypy = range(N)
Xnp = np.arange(N)
Ynp = np.arange(N)

We will use the magic command `%timeit` to time how long it takes to execute.

In [ ]:
%timeit _ = [Xpy[i] + Ypy[i] for i in range(N)]

In [ ]:
%timeit _ = Xnp + Ynp

| prefix | symbol | value | 
| ------ | ------ | ----- | 
| deci   | d      | $10^{-1}$ |
| centi  | c      | $10^{-2}$ |
| milli  | m      | $10^{-3}$ |
| micro  | μ      | $10^{-6}$ | 
| nano   | n      | $10^{-9}$ | 
| pico   | p      | $10^{-12}$ |
| atto   | a      | $10^{-18}$ | 

[Attosecond physics 🤯](https://en.wikipedia.org/wiki/Attosecond_physics)

The magnitude difference is 1000 times.

## Indexing with `ndarray`

In Python counting starts with $0$, so it can be confusing.

Sometimes I refer to the first entry of a list as the 'first' entry, and sometimes I refer to it as the 'zeroth' entry. 

This is confusing, but I try to correct myself and use 'zeroth'.

For the other entries, I generally match what Python would use. 

The simplest example is the $1$-dimensional array, so let's work with that. 

In [ ]:
a1 = np.random.randint(100, size=6)
print(a1)

We access the entries of `a1` (and any $1$-dimensional array) with a single integer. 

In our example the integers $\{0, 1, 2, 3, 4, 5\}$ are suitable.

In [ ]:
a1[0]

In [ ]:
a1[2]

In [ ]:
a1[5]

In [13]:
# a1[6]       # naughty naughty

Nonnegative integers are used to access entries from left to right.

Negative integers are used to access entries from right to left.

For our example, we can also use the integers from $\{-1,-2,-3,-4,-5,-6\}$.

In [ ]:
a1[-1]

In [ ]:
a1[-3]

In [ ]:
a1[-6]

In [17]:
# a[-10]        # naughty naughty

Every $1$-dimensional `ndarray` of length $N$ can be indexed with the integers 
$$
    \{-N,\ -N+1,\ \dots,\ -1,\ 0,\ 1,\ \dots,\ N-2,\ N-1\} . 
$$

**Quick Note.** You can determine the length of an array `a` by `len(a)`.

We can surgically change one entry of the array

In [ ]:
a1[3] = 137
print(a1)

We can take this ideas and generalize to higher dimensional arrays.

Let's see the leap from $1$ to $2$ dimensions.

In [ ]:
a2 = np.random.randint(100, size=(3, 9))
print(a2)

Entries are indexed the same way we index matrices. 

For example, the $(i,j)$ entry of a matrix lies in the $i$th row and $j$th column. 

We access entries by pairs of integers.

In [ ]:
a2[0, 0]

In [ ]:
a2[0, 4]

In [ ]:
a2[2, 6]

We can think of the first entry as taking an integer from 
$$
    \{-3, -2, -1, 0, 1, 2\}
$$

and the second entry from
$$
    \{-9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8\}.
$$

In [ ]:
a2[-2, 8]

Moving to higher dimensions, the same ideas apply.

In [ ]:
a5 = np.random.randint(100, size=(4, 3, 5, 7, 2))
print(a5)

In [ ]:
a5[0, 0, 0, 0, 0]

There are a few more indexing tricks, but this covers most of what one would do.

If you want to learn more, check out the [documentation](https://numpy.org/doc/stable/user/basics.indexing.html).

## Slicing arrays

A slice of an array is a subarray, which can be lower-dimensional than the original.

### One-dimensional slices

In some sense, this is the most boring, but it's also the easiest to understand.

Accessing entries was done by `a1[k]` for some $k$.

We will take a range of entries from `a1`.

In [ ]:
print(a1)

In [ ]:
print(a1[1:5])

The syntax `a1[i:j]` takes all entries from $i$ to (and including) $j-1$. 

In [ ]:
print(a1[0:6])

In [ ]:
print(a1[-6:-1])

There's a *third* argument you can use.

In [ ]:
a1 = np.arange(20)
print(a1)

In [ ]:
print(a1[0:20:2])

In [ ]:
print(a1[:5])
print(a1[5:])
print(a1[:])

In [ ]:
print(a1[::2])
print(a1[::])
print(a1[::-1])

### Jumping to $3$ dimensions

It might be helpful to visualize a $3$-dimensional array as a rectangular prism of data.

The following is an illustration of a $(5\times 6\times 4)$-array.

![](imgs/multiway_array.png)

In [ ]:
a3 = np.random.randint(10, size=(5, 6, 4))
print(a3)

'Slicing' is an operation on arrays that yield 'subarrays'. 

For example, here are a few slices of the above array:

![](imgs/sliced.png)

In [ ]:
for k in range(4):
    print(a3[:, :, k])
    print()

In [ ]:
print(a3[0])
print()
print(a3[0, :, :])

## Creating copies

This might seem silly, but it is important. 

Here's a problem without an error. 

In [ ]:
a2 = np.random.randint(10, size=(3, 4))
print(a2)

In [ ]:
b2 = a2[1:, 1:]
print(b2)

In [ ]:
b2[0, 0] = -1
print(b2)

In [ ]:
print(a2)

This might not be intended. If you want to edit `b2` independently of `a2`, they need to be independent of each other.

We can do this by the `copy` method.

In [ ]:
b2 = a2[1:, 1:].copy()
print(b2)

In [ ]:
b2[0, 0] = 42
print(b2)
print()
print(a2)

Be careful out there.

## Reshaping

We can reshape arrays into other appropriate sizes.

In [ ]:
a1 = np.arange(10)
a2 = a1.reshape(2, 5)
print(a1)
print()
print(a2)

In [ ]:
a2[0, 0] = 10
print(a2)

In [ ]:
print(a1)

Therefore `reshape` is not making a copy in general. Keep that in mind. 

The shapes are all distinct:
$$
    (n),\; (1, n),\; (n, 1),\; (n, 1, 1),\; (1, 1, n, 1, 1, 1),\; \text{etc}.
$$

In [46]:
a1 = np.arange(5)
a2_r = np.arange(5).reshape(1, 5)
a2_c = np.arange(5).reshape(5, 1)
a4 = np.arange(5).reshape(1, 5, 1, 1)

In [ ]:
print(a1)
print(a2_r)
print(a2_c)
print(a4)

### `newaxis`

A common enough reshape occurs when one takes a $1$-dimensional array and converts it to either a row or column vector. 

`reshape` works here, but so does `newaxis`.

In [ ]:
print(a1)

In [ ]:
print(a1[np.newaxis, :])

In [ ]:
print(a1[:, np.newaxis])

In [ ]:
print(a1[np.newaxis, :, np.newaxis, np.newaxis])

## Concatenating

As usual, with $1$-dimensional arrays the notion of concatenation is simple.

In [52]:
a1 = np.arange(5)
b1 = np.arange(20, 25)
c1 = np.arange(9, 2, -2)

In [ ]:
np.concatenate([a1, b1, c1])

For higher dimensions, concatenation gets confusing. 

![](imgs/confused_thinking.png)

**We concatenate *along* an axis.**

For $1$-dimensional arrays, there is only one axis, so it is unambiguous. 

For $2$-dimensional arrays, you have horizontal and vertical. 

For an $n$-dimensional array, there are $n$ axes labeled $0$, $1$, up to $n-1$.

When we indexed an entry, we gave specific coordinates to the *axes*.

So `a3[i, j, k]` takes the entry in the $i^{th}$ position on axis 0, the $j^{th}$ position on axis 1, and the $k^{th}$ position on axis 2. 

#### Matrices

Since `a2[i, j]` takes the entry in row $i$ and column $j$, we know 
- axis 0 : rows
- axis 1 : columns

Say it again:

**We concatenate *along* an axis.**

If we concatenate *along* axis 0, we concatenate along the rows. This is a *vertical* concatenation.

In [ ]:
a2 = np.arange(12).reshape(3, 4)
b2 = np.arange(42, 50).reshape(2, 4)
print(a2)
print()
print(b2)

In [ ]:
print(np.concatenate([a2, b2], axis=0))

If we concatenate *along* axis 1, we concatenate along the columns. This is a *horizontal* concatenation.

In [ ]:
a2 = np.arange(12).reshape(3, 4)
b2 = np.arange(42, 48).reshape(3, 2)
print(a2)
print()
print(b2)

In [ ]:
print(np.concatenate([a2, b2], axis=1))

I don't really want to go higher. 

## Splitting

The function `split` is, in some sense, the inverse to `concatenate`, so we'll go fast.

In [ ]:
a1 = np.arange(8)
print(a1)

In [ ]:
print(np.split(a1, 4))

In [ ]:
print(np.split(a1, [3, 4]))

Running `np.split(a1, k)` for an integer $k$ returns `a1` split into *equal* sized arrays of length $k$.

If $k$ is not a divisor of `len(a1)`, an error is raised.

Running `np.split(a1, [i, j, k])` with $i < j < k$, all three integers, then 
```python
a1[:i],  a1[i:j],  a1[j:k],  a1[k:]
```

is returned.

The idea generalizes to higher dimensions using the keyword argument `axis`. 

As with concatenation, splitting happens *along* a given axis. 

## Exercises
1. Starting with a $1$-dimensional array of length $60$,
   reshape it into a $3$-dimensional array with dimensions
   of sizes $5$, $4$ and $3$, respectively.
2. Then split the array along the second dimenson,
   the one of size $4$, into two halves.
3. What does `np.newaxis` mean, and what is it used for?